In [1]:
!pip install replicate duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 77.6 MB/s eta 0:00:00


In [3]:
import replicate
from duckduckgo_search import DDGS

# 🔐 IBM Granite on Replicate
REPLICATE_API_TOKEN = "r8_0t7oSmbYTPDemQOdPHOrvP6Aq4OrhDr4F0fGW"
client = replicate.Client(api_token=REPLICATE_API_TOKEN)

# 🔎 Optional: DuckDuckGo search fallback
def search_duckduckgo(query):
    with DDGS() as ddgs:
        results = ddgs.text(query, max_results=3)
        return "\n".join([f"- {r['title']}: {r['href']}" for r in results])

# 🧠 LLM query function with system prompt injection
def query_replicate(prompt, model="ibm-granite/granite-3.3-8b-instruct"):
    output = client.run(model, input={"prompt": prompt, "max_new_tokens": 300})
    return "".join(output)

# 🎓 Agent 1: Admission Agent
def admission_agent(prompt):
    system_prompt = (
        "You are an expert college admission advisor. "
        "Provide detailed information on admission processes, eligibility, deadlines, and application tips."
    )
    return query_replicate(f"{system_prompt}\n\nUser: {prompt}")

# 📚 Agent 2: Course and Program Advisor
def course_agent(prompt):
    system_prompt = (
        "You are a knowledgeable academic counselor. "
        "Help users understand different courses, degree programs, majors, and specializations offered by colleges."
    )
    return query_replicate(f"{system_prompt}\n\nUser: {prompt}")

# 💰 Agent 3: Scholarship and Financial Aid Advisor
def scholarship_agent(prompt):
    system_prompt = (
        "You are a financial aid specialist. "
        "Provide guidance about scholarships, grants, bursaries, eligibility, and application procedures."
    )
    return query_replicate(f"{system_prompt}\n\nUser: {prompt}")

# 🏫 Agent 4: Campus Life Advisor
def campus_life_agent(prompt):
    system_prompt = (
        "You are a college campus life expert. "
        "Inform users about campus facilities, hostels, clubs, sports, and extracurricular activities."
    )
    return query_replicate(f"{system_prompt}\n\nUser: {prompt}")


In [4]:
def college_admission_router(user_prompt):
    admission_keywords = ["admission", "apply", "application", "deadline", "eligibility", "requirements"]
    course_keywords = ["course", "program", "degree", "major", "subject", "specialization"]
    scholarship_keywords = ["scholarship", "financial aid", "funding", "grant", "bursary", "assistantship"]
    campus_life_keywords = ["campus", "hostel", "facilities", "clubs", "events", "sports", "library"]

    prompt_lower = user_prompt.lower()

    if any(word in prompt_lower for word in admission_keywords):
        return admission_agent(user_prompt)
    elif any(word in prompt_lower for word in course_keywords):
        return course_agent(user_prompt)
    elif any(word in prompt_lower for word in scholarship_keywords):
        return scholarship_agent(user_prompt)
    elif any(word in prompt_lower for word in campus_life_keywords):
        return campus_life_agent(user_prompt)
    else:
        # default fallback agent
        return "🤖 Could you please clarify your question? Are you asking about admissions, courses, scholarships, or campus life?"


In [7]:
import gradio as gr

def respond_fn(user_message, history):
    bot_message = college_admission_router(user_message)
    history = history or []
    history.append((user_message, bot_message))
    return history, ""

with gr.Blocks() as demo:
    gr.Markdown("## 🎓 College Admission Agent AI Chatbot")
    chatbot = gr.Chatbot()
    user_input = gr.Textbox(placeholder="Type your question here...", label="Your Question")
    submit_btn = gr.Button("Send")

    submit_btn.click(respond_fn, inputs=[user_input, chatbot], outputs=[chatbot, user_input])
    user_input.submit(respond_fn, inputs=[user_input, chatbot], outputs=[chatbot, user_input])

demo.launch()


/tmp/ipython-input-3057128275.py:11: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3ece80d0077cea75ca.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
